## Imports

In [212]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [213]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Aug. 12th Aug. 19th


In [214]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [215]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [216]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [217]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [8]:
# # Filter dates between. If only using data looking back one week, this shouldn't change the df
# df = df[(df['SOLD DATE'] >= '2024-06-10') & (df['SOLD DATE'] <= '2024-06-17')]

In [218]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
42,PAST SALE,2024-08-19,Condo/Co-op,701 Brickell Key Blvd #1206,Miami,FL,33131.0,775000.0,2.0,2.0,COURVOISIER COURTS CONDO,1288.0,NaN,1997.0,NaN,602.0,1614.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/701-Brickell-K...,MARMLS,A11604510,N,Y,25.767951,-80.186447
73,PAST SALE,2024-08-19,Condo/Co-op,7757 SW 86th St Unit C-316,Miami,FL,33143.0,265000.0,1.0,1.0,KINGS CREEK SO CONDO,715.0,NaN,1969.0,NaN,371.0,410.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/7757-SW-86th-S...,MARMLS,A11562183,N,Y,25.691529,-80.319893
13,PAST SALE,2024-08-19,Condo/Co-op,17021 N Bay Rd #906,Sunny Isles Beach,FL,33160.0,410000.0,2.0,2.5,PLAZA OF AMERICAS CONDO P,1230.0,NaN,1979.0,NaN,333.0,958.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11614445,N,Y,25.932790,-80.124335
3,PAST SALE,2024-08-17,Condo/Co-op,9200 W Bay Harbor Dr Unit 1A,Bay Harbor Islands,FL,33154.0,1122000.0,2.0,2.0,9200 W BAY HARBOR CONDO,2200.0,NaN,1959.0,NaN,510.0,867.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/9...,MARMLS,A11642965,N,Y,25.882391,-80.131876
15,PAST SALE,2024-08-16,Condo/Co-op,17341 NW 7th Ave #605,Miami Gardens,FL,33169.0,280000.0,2.0,2.5,SOLABELLA CONDO,1140.0,NaN,2007.0,NaN,246.0,270.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/17341-NW-7th-A...,MARMLS,A11597710,N,Y,25.935134,-80.210927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,PAST SALE,2024-08-12,Condo/Co-op,9310 Fontainebleau Blvd #312,Miami,FL,33172.0,237000.0,1.0,1.0,The Fountains OF Fontainebleu NO 1,690.0,NaN,1972.0,NaN,343.0,332.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/9310-Fontaineb...,Beaches MLS,RX-10997598,N,Y,25.771339,-80.347174
72,PAST SALE,2024-08-12,Condo/Co-op,10700 NW 66th St #309,Doral,FL,33178.0,470000.0,3.0,2.0,CAPTIVA CONDO F,1710.0,NaN,2002.0,NaN,275.0,1067.0,Sold,NaN,NaN,https://www.redfin.com/FL/Doral/10700-NW-66th-...,MARMLS,A11585098,N,Y,25.829136,-80.372262
76,PAST SALE,2024-08-12,Condo/Co-op,7707 Camino Real Unit B-405,Miami,FL,33143.0,290000.0,2.0,1.0,VILLAGE OF KINGS CREEK CO,804.0,NaN,1972.0,NaN,361.0,465.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/7707-Camino-Re...,MARMLS,A11569723,N,Y,25.694331,-80.319079
79,PAST SALE,2024-08-12,Condo/Co-op,16909 N Bay Rd #116,Sunny Isles Beach,FL,33160.0,330000.0,2.0,2.0,PLAZA OF AMERICAS CONDO P,988.0,NaN,1979.0,NaN,334.0,700.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11545245,N,Y,25.930951,-80.125012


In [219]:
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Collect Agent Information

In [220]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [221]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [222]:
response_df = pd.DataFrame(response_list)

In [223]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [237]:
df_top_ten = merged_df

In [186]:
# columns_to_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','LOCATION',
#                    'LOT SIZE','DAYS ON MARKET',
#                   'HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','URL',
#                   'FAVORITE','INTERESTED','url','list_broker_email_1',
#                    'list_broker_email_2','buy_broker_email_1','buy_broker_email_2','latitude','longitude',
#                   'latest_price','num_full_baths','num_partial_baths','price_per_sq_ft','sq_ft','year_built',
#                   'assembled_address','data_source_name','mls_id']

# chart_df = merged_df.drop(columns=columns_to_drop)

# chart_df = chart_df.rename(columns={'SOLD DATE':'Close Date','ADDRESS':'Address','BEDS':'Beds','BATHS':'Baths',
#                          'PRICE':'Price','SQUARE FEET':'Square Feet','YEAR BUILT':'Year Built',
#                         '$/SQUARE FEET':'PSF','list_agent_name_1':'List Agent 1',
#                          'list_broker_name_1':'List Broker 1','SOURCE':'Source',
#                         'list_agent_name_2':'List Agent 2', 'list_broker_name_2':'List Broker 2',
#                         'buy_agent_name_1':'Buy Agent 1','buy_broker_name_1':'Buy Broker 1',
#                         'buy_agent_name_2':'Buy Agent 2','buy_broker_name_2':'Buy Broker 2'})

# merged_df['ADDRESS'] = merged_df['ADDRESS'] + ' ' + merged_df['CITY']

## Current Week's Values

In [225]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

110


In [226]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$694,298


In [227]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$457


In [228]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$76,372,819


In [14]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [229]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
92
Input Previous Week Condo Average Sales Price:
678200
Input Previous Week Condo Average PSF:
530
Input Previous Week Condo Sales total (ex: 198_000_000)
105000000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [241]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [242]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [243]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [244]:
pd.set_option('display.max_columns',None)

In [245]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-08-16,Condo/Co-op,2669 S Bayshore Dr Unit LPH-N,Miami,FL,33133.0,19500000.0,6.0,7.0,DINNER KEY,10065.0,NaN,2016.0,NaN,1937.0,12929.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11433836,N,Y,25.730657,-80.237098,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Chad Carroll,"Compass Florida, LLC",Lisandra Martinez,"Compass Florida, LLC",Malgozata Makovska Fuller,One Sotheby's International Re,None,None,1,orange


In [252]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [253]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-08-16,Condo/Co-op,2669 S Bayshore Dr Unit LPH-N,Miami,FL,33133.0,19500000.0,6.0,7.0,DINNER KEY,10065.0,NaN,2016.0,NaN,1937.0,12929.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11433836,N,Y,25.730657,-80.237098,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Chad Carroll,"Compass Florida, LLC",Lisandra Martinez,"Compass Florida, LLC",Malgozata Makovska Fuller,One Sotheby's International Re,None,None,1,orange
1,PAST SALE,2024-08-14,Condo/Co-op,17141 Collins Ave #3902,Sunny Isles Beach,FL,33160.0,3070000.0,2.0,2.5,MUSE CONDO,2044.0,NaN,2018.0,NaN,1502.0,4329.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11533135,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Sarah Gozlan,"G Realty Group, LLC.",None,None,Maria Papaspirou,Dream Life Realty,None,None,2,blue
2,PAST SALE,2024-08-14,Condo/Co-op,101 Sunrise Dr Unit A-404,Key Biscayne,FL,33149.0,2730000.0,3.0,3.5,101 CONDOMINIUM,2201.0,NaN,2015.0,NaN,1240.0,2400.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/101-Sun...,MARMLS,A11593861,N,Y,25.690289,-80.163283,https://www.redfin.com/FL/Key-Biscayne/101-Sun...,Grehyni Narvaez,Avanti Way Realty LLC,None,None,Grehyni Narvaez,Avanti Way Realty LLC,None,None,3,blue
3,PAST SALE,2024-08-12,Condo/Co-op,15901 Collins Ave #2702,Sunny Isles Beach,FL,33160.0,1850000.0,3.0,3.5,TDR TOWER II CONDO,1979.0,NaN,2008.0,NaN,935.0,2426.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11625053,N,Y,25.923693,-80.121772,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,Maria Ramos,"Brokers, LLC",None,None,Wictor Arraes,Xcellence Realty,None,None,4,blue
4,PAST SALE,2024-08-13,Condo/Co-op,16699 Collins Ave #4101,Sunny Isles Beach,FL,33160.0,1450000.0,2.0,2.5,La Perla Condominium,1623.0,NaN,2007.0,NaN,893.0,1330.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Beaches MLS,F10444082,N,Y,25.930187,-80.121374,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Joseph Sigal,London Foster Realty,None,London Foster Realty,Olga Zaurova,"Big International Realty, Inc.",None,None,5,blue
5,PAST SALE,2024-08-12,Condo/Co-op,1331 Brickell Bay Dr #2508,Miami,FL,33131.0,1375000.0,2.0,2.0,JADE RESIDENCES AT BRICKE,1529.0,NaN,2004.0,NaN,899.0,2117.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1331-Brickell-...,MARMLS,A11600843,N,Y,25.760000,-80.189163,https://www.redfin.com/FL/Miami/1331-Brickell-...,Carolina Arenas,"Compass Florida, LLC.",None,None,Robert Quintero,International Real Estate Investors,None,None,6,blue
6,PAST SALE,2024-08-15,Condo/Co-op,1000 Brickell Plz #1915,Miami,FL,33131.0,1315000.0,2.0,2.5,BRICKELL FLATIRON CONDO,1334.0,NaN,2019.0,NaN,986.0,2031.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11503334,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Maria Mosquera,"ON Florida Realty, LLC.",Valentina Delgado,"ON Florida Realty, LLC.",Evan Schechtman,Blackbook Properties,Jessica Turner,Blackbook Properties,7,blue
7,PAST SALE,2024-08-12,Condo/Co-op,9240 W Bay Harbor Dr Unit 3A,Bay Harbor Islands,FL,33154.0,1310000.0,2.0,2.5,THE MONTEREY CONDO,2103.0,NaN,1969.0,NaN,623.0,1210.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bay-Harbor-Islands/9...,MARMLS,A11538085,N,Y,25.882644,-80.132214,https://www.redfin.com/FL/Bay-Harbor-Islands/9...,Paul Basile,Douglas Elliman,None,None,Clarissa Valdivia,Elite International Realty Inc,None,None,8,blue
8,PAST SALE,2024-08-14,Condo/Co-op,540 West Ave #2014,Miami Beach,FL,33139.0,1265000.0,2.0,2.

In [254]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [255]:
m.save('index.html')

## Data snagger

In [256]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [257]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2024-08-16,Condo/Co-op,2669 S Bayshore Dr Unit LPH-N,Miami,FL,33133.0,19500000.0,6.0,7.0,DINNER KEY,10065.0,NaN,2016.0,NaN,1937.0,12929.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11433836,N,Y,25.730657,-80.237098,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Chad Carroll,"Compass Florida, LLC",Lisandra Martinez,"Compass Florida, LLC",Malgozata Makovska Fuller,One Sotheby's International Re,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [258]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_08192024


# CREATE TEMPLATE 

In [259]:
muni_set = set(df_top_ten['CITY'])

In [260]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [261]:
df_top_ten.reset_index(inplace=True,drop=True)

In [262]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [263]:
top_sale

'DINNER KEY at 2669 S Bayshore Dr Unit LPH-N in Miami'

In [264]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [265]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

## RESUME HERE

In [266]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: DINNER KEY closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,250,000M to $19,500,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami Beach, Key Biscayne, North Miami Beach, Miami, Sunny Isles Beach, Bay Harbor Islands

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 110 condo sales totaling $76,372,819 million from Aug. 12th to Aug. 19th. The previous week, brokers closed 92 condo sales totaling $105,000,000.

Last week’s units sold for an average of $694,298, higher than the $678,200 sales average from the 

In [267]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [268]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [272]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

Listing agents: Chad Carroll with Compass Florida, LLC and Lisandra Martinez with Compass Florida, LLC


In [350]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}")

print(story_checklist)


HED: DINNER KEY closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,250,000M to $19,500,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami Beach, Key Biscayne, North Miami Beach, Miami, Sunny Isles Beach, Bay Harbor Islands

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 110 condo sales totaling $76,372,819 million from Aug. 12th to Aug. 19th. The previous week, brokers closed 92 condo sales totaling $105,000,000.

Last week’s units sold for an average of $694,298, higher than the $678,200 sales average from the 

In [351]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Miami/2669-S-Bayshore-Dr-33133/unit-LPH-N/home/190088363


In [355]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Sunny-Isles-Beach/17141-Collins-Ave-33160/unit-3902/home/146767561


In [353]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Aventura/17111-Biscayne-Blvd-33160/unit-1801/home/79132989


In [354]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami-Beach/540-West-Ave-33139/unit-2014/home/43341826


In [284]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami/2669-S-Bayshore-Dr-33133/unit-LPH-N/home/190088363


In [285]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Bay-Harbor-Islands/9240-W-Bay-Harbor-Dr-33154/unit-3A/home/42904418


In [286]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-08-16,Condo/Co-op,2669 S Bayshore Dr Unit LPH-N,Miami,FL,33133.0,19500000.0,6.0,7.0,DINNER KEY,10065.0,NaN,2016.0,NaN,1937.0,12929.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,MARMLS,A11433836,N,Y,25.730657,-80.237098,https://www.redfin.com/FL/Miami/2669-S-Bayshor...,Chad Carroll,"Compass Florida, LLC",Lisandra Martinez,"Compass Florida, LLC",Malgozata Makovska Fuller,One Sotheby's International Re,None,None,1,orange,DINNER KEY at 2669 S Bayshore Dr Unit LPH-N in...
1,PAST SALE,2024-08-14,Condo/Co-op,17141 Collins Ave #3902,Sunny Isles Beach,FL,33160.0,3070000.0,2.0,2.5,MUSE CONDO,2044.0,NaN,2018.0,NaN,1502.0,4329.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,MARMLS,A11533135,N,Y,25.934939,-80.120918,https://www.redfin.com/FL/Sunny-Isles-Beach/17...,Sarah Gozlan,"G Realty Group, LLC.",None,None,Maria Papaspirou,Dream Life Realty,None,None,2,blue,MUSE CONDO at 17141 Collins Ave #3902 in Sunny...
2,PAST SALE,2024-08-14,Condo/Co-op,101 Sunrise Dr Unit A-404,Key Biscayne,FL,33149.0,2730000.0,3.0,3.5,101 CONDOMINIUM,2201.0,NaN,2015.0,NaN,1240.0,2400.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/101-Sun...,MARMLS,A11593861,N,Y,25.690289,-80.163283,https://www.redfin.com/FL/Key-Biscayne/101-Sun...,Grehyni Narvaez,Avanti Way Realty LLC,None,None,Grehyni Narvaez,Avanti Way Realty LLC,None,None,3,blue,101 CONDOMINIUM at 101 Sunrise Dr Unit A-404 i...
3,PAST SALE,2024-08-12,Condo/Co-op,15901 Collins Ave #2702,Sunny Isles Beach,FL,33160.0,1850000.0,3.0,3.5,TDR TOWER II CONDO,1979.0,NaN,2008.0,NaN,935.0,2426.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,MARMLS,A11625053,N,Y,25.923693,-80.121772,https://www.redfin.com/FL/Sunny-Isles-Beach/15...,Maria Ramos,"Brokers, LLC",None,None,Wictor Arraes,Xcellence Realty,None,None,4,blue,TDR TOWER II CONDO at 15901 Collins Ave #2702 ...
4,PAST SALE,2024-08-13,Condo/Co-op,16699 Collins Ave #4101,Sunny Isles Beach,FL,33160.0,1450000.0,2.0,2.5,La Perla Condominium,1623.0,NaN,2007.0,NaN,893.0,1330.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Beaches MLS,F10444082,N,Y,25.930187,-80.121374,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Joseph Sigal,London Foster Realty,None,London Foster Realty,Olga Zaurova,"Big International Realty, Inc.",None,None,5,blue,La Perla Condominium at 16699 Collins Ave #410...
5,PAST SALE,2024-08-12,Condo/Co-op,1331 Brickell Bay Dr #2508,Miami,FL,33131.0,1375000.0,2.0,2.0,JADE RESIDENCES AT BRICKE,1529.0,NaN,2004.0,NaN,899.0,2117.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1331-Brickell-...,MARMLS,A11600843,N,Y,25.760000,-80.189163,https://www.redfin.com/FL/Miami/1331-Brickell-...,Carolina Arenas,"Compass Florida, LLC.",None,None,Robert Quintero,International Real Estate Investors,None,None,6,blue,JADE RESIDENCES AT BRICKE at 1331 Brickell Bay...
6,PAST SALE,2024-08-15,Condo/Co-op,1000 Brickell Plz #1915,Miami,FL,33131.0,1315000.0,2.0,2.5,BRICKELL FLATIRON CONDO,1334.0,NaN,2019.0,NaN,986.0,2031.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1000-Brickell-...,MARMLS,A11503334,N,Y,25.764264,-80.192725,https://www.redfin.com/FL/Miami/1000-Brickell-...,Maria Mosquera,"ON Florida Realty, LLC.",Valentina Delgado,"ON Florida Realty, LLC.",Evan Schechtman,Blackbook Properties,Jessica Turner,Blackbook Properties,7,blue,BRICKELL FLATIRON CONDO at 1000 Brickell Plz #...
7,PAST SALE,2024-08-12,Condo/Co-op,9240 W Bay Harbor Dr Unit 3A,Bay Harbor Islands,FL,33154.0,1310000.0,2.0,2.5,THE MONTEREY CONDO,2103.0,NaN,1969.0,NaN,623.0

## Time on Market Calculator

In [356]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 2, 14) ## List (Earlier) date
date2 = datetime(2024, 8, 14) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

182


## Clean CSV for Datawrapper Chart

In [382]:
chart_df = df_top_ten

In [383]:
chart_df = chart_df.fillna(" ")

In [384]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [385]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [386]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [387]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [388]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [390]:
chart_df.to_csv("WCS_Chart_08_19_24.csv")